In [ ]:
# dependencies import
from common_dependencies import *
from itertools import chain
import logging
logger = logging.getLogger('main.make_pred.ipynb')

In [ ]:
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

# paths for files with original data
RUNS = [2]
XSHIFT = 200

MODEL_VER = 14
MODEL_NUM = 11
EPOCH = 60

model_pathes = [path for path in pathlib.Path(f'networks/CNN').rglob(f'*.keras') 
                     if re.search(rf'id=v0*{MODEL_VER}n0*{MODEL_NUM}.*epoch={EPOCH}*',path.name)]

if len(model_pathes) == 1:
    PATH_TO_MODEL = model_pathes[0]
else:
    print(model_pathes)
    raise ValueError('Few or none model have been found instead of one')

print(f'{PATH_TO_MODEL=}')

In [ ]:
# Загрузка модели
load_model = keras.models.load_model(PATH_TO_MODEL)
CROP_SIZE = load_model.inputs[0].shape[1]
CROP_STEPS = [16] #[CROP_SIZE, CROP_SIZE//2, CROP_SIZE//4, CROP_SIZE//8]
print(f'{CROP_SIZE=}')
print(f'{CROP_STEPS=}')
'''print(load_model.summary())
tf.keras.utils.plot_model(
    load_model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)''';

# Новый алгоритм построения предсказаний

In [ ]:
'''def build_add_remove_1_prediction(extend_size_result_df: pd.DataFrame, orig_shape: tuple[int,int], 
                                  crop_size: int, crop_step: int, res: np.ndarray, tholds=[0.2,0.4,0.6,0.8], **kvargs):
    """
    The algorithm that will iterate through the 
    ex_x_df size empty pandas.DataFrame like it is being dividing by crops
    of given params and if the current prediction greater or equal to the current 
    thold from the tholds, so all cells of the empty dataframes that refer to the current
    crop will be incremented by 1, otherwise decremented by 1.

    Parameters
    ----------
    extend_size_result_df: pandas.DataFrame
        The empty dataframe of same shape as extended for prediction 
        and crop dividing one for build the prediction map on base to. 
    orig_shape: tuple[int,int]
        The shape of ex_x_df before extending.
    res: np.ndarray
        The flat array of the model predictions for all crops of the
        ex_x_df preprocessed before.
    tholds: iterable
        The thresholds to use in the algorithm.
    """
    res_it = iter(res)
    for i in range(0, extend_size_result_df.shape[0] - crop_size + 1, crop_step):
        for j in range(0,  extend_size_result_df.shape[1] - crop_size + 1, crop_step):
            temp_add = next(res_it)
            for thold in tholds:
                if temp_add >= thold:
                    extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                        extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: x+1)
                else:
                    extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                        extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: x-1)
                    
            extend_size_result_df = pd.DataFrame(data=dw.normalize_data(extend_size_result_df.to_numpy()), 
                                         index=extend_size_result_df.index, 
                                         columns=extend_size_result_df.columns)
            orig_size_result_df = extend_size_result_df.iloc[crop_size-1:, crop_size-1:].iloc[:orig_shape[0],:orig_shape[1]]

            info = {'crop_size': crop_size, 'crop_step': crop_step, 'thold': thold}
            
            yield orig_size_result_df, extend_size_result_df, {**info, **kvargs}''';

In [ ]:
'''def build_add_to_array_prediction(extend_size_result_df: pd.DataFrame, orig_shape: tuple[int,int], 
                                  crop_size: int, crop_step: int, res: np.ndarray, **kvargs): 
    """
    The algorithm that will iterate through the 
    ex_x_df size empty pandas.DataFrame like it is being dividing by crops
    of given params and add every prediction value to arrays in each cell that 
    refer to the current crop.

    Parameters
    ----------
    extend_size_result_df: pandas.DataFrame
        The empty dataframe of same shape as extended for prediction 
        and crop dividing one for build the prediction map on base to. 
    orig_shape: tuple[int,int]
        The shape of ex_x_df before extending.
    res: np.ndarray
        The flat array of the model predictions for all crops of the
        ex_x_df preprocessed before.
    """
    extend_size_result_df.map(lambda x: list())
    res_it = iter(res)
    for i in range(0, extend_size_result_df.shape[0] - crop_size + 1, crop_step):
        for j in range(0,  extend_size_result_df.shape[1] - crop_size + 1, crop_step):
            temp_add = next(res_it)
            extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
               extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: [*x,temp_add])
            
    orig_size_result_df = extend_size_result_df.iloc[crop_size-1:, crop_size-1:].iloc[:orig_shape[0],:orig_shape[1]]

    info = {'crop_size': crop_size, 'crop_step': crop_step, 
            'xshift': xshift, 'thold': thold}
    
    yield orig_size_result_df, extend_size_result_df, {**info, **kvargs}''';

In [ ]:
'''def build_pred_map(ex_x_df, orig_shape: tuple[int,int], res, crop_size: int, crop_step: int, method: str='add_remove_1', **kvargs):
    """
    Build prediction map for the ex_x_df data and res_it predictions iterator with
    defined crop step and crop size by method defined by the method param.

    Parameters
    ----------
    ex_x_df: pandas.DataFrame
        The extended for prediction and crop dividing read data 
        for build the prediction map on base to. The prediction 
        map should be the pandas.DataFrame of the same shape and
        column and index names as the ex_x_df.
    orig_shape: tuple[int,int]
        The shape of ex_x_df before extending.
    res: np.ndarray
        The flat array of the model predictions for all crops of the
        ex_x_df preprocessed before.
    crop_size: int
        The crop size used to preprocess read data.
    crop_step: int
        The crop step used to preprocess read data.
    method: str
        Which of two method for predictions aggregation to use.
        May be either 'add_remove_1' or 'add_to_array'.
        If 'add_remove_1' the algorithm will iterate through the 
        ex_x_df size empty pandas.DataFrame like it is being dividing by crops
        of given params and if the current prediction greater or equal to the current 
        thold from the tholds, so all cells of the empty dataframes that refer to the current
        crop will be incremented by 1, otherwise decremented by 1.
        If 'add_to_array' the algorithm will iterate through the 
        ex_x_df size empty pandas.DataFrame like it is being dividing by crops
        of given params and add every prediction value to arrays in each cell that 
        refer to the current crop.
    **kvargs
        Any additional params that may be defined in the build_add_remove_1_prediction() or
        build_add_to_array_prediction().
    """
    if not method in ('add_remove_1','add_to_array'):
        raise ValueError('The method param should be one of [\'add_remove_1\',\'add_to_array\']')
        
    extend_size_result_df = pd.DataFrame(data=0.0, 
                             columns=ex_x_df.columns, 
                             index=ex_x_df.index)
    
    if method == 'add_remove_1':
        return build_add_remove_1_prediction(extend_size_result_df, orig_shape, crop_size, crop_step, res, method=method, **kvargs)
    else:
        return build_add_to_array_prediction(extend_size_result_df, orig_shape, crop_size, crop_step, res, method=method, **kvargs)''';

In [ ]:
'''def get_xshifts_data_generator(model, x_df: pd.DataFrame, y_df: pd.DataFrame, xshifts, crop_step: int, buffer: int=15_000, **kvargs):
    """
    For any crop step there are crop_step options of result prediction map due
    to different start point. So the function enumerate all crop_sizes and xshifts.
    Futhermore data preprocessing deffers for all combinations. Then the function
    calls build_pred_map() to build prediction.

    Parameters
    ----------
    model: tensorflow.keras.Model
        The tensorflow model to use in prediction.
    x_df: pandas.DataFrame
        The read data for passing into the model inputs.
    y_df: pandas.DataFrame
        The read data for cheching model predictions quality.
    xshifts: iterable
        The any flat iterable with all xshifts for the crop_step.
    crop_step: int
        The crop step to use in data preprocessing and prediction
        map building.
    buffer: int
        How many crops the PC can store in RAM at one moment. The bigger - 
        the faster it works.
    **kvargs
        Any additional params that may be defined in the build_pred_map().
    """
    crop_size = model.inputs[0].shape[1]
    for xshift in xshifts:
        x_df = dw.roll_df(x_df, xshift, 1)
        y_df = dw.roll_df(y_df, xshift, 1)
        orig_shape = x_df.shape
    
        ex_x_df = dw.extend_df_for_prediction(x_df, crop_size)
        ex_x_df = dw.match_df_for_crops_dividing(ex_x_df, crop_size, crop_step)
        ex_y_df = dw.extend_df_for_prediction(y_df, crop_size)
        ex_y_df = dw.match_df_for_crops_dividing(ex_y_df, crop_size, crop_step)
        
        x_arr = dw.df_to_numpy(ex_x_df)
        y_arr = ex_y_df.to_numpy()

        # check model inputs and outputs quantity
        if len(model.inputs) > 2:
            raise ValueError('There are no implementation for handling model with 3 inputs')
        if len(model.outputs) > 2:
            raise ValueError('There are no implementation for handling model with 3 inputs')

        # if model has one or two inputs prepare data differently
        x_crops_gen = None
        if len(model.inputs) == 1:
            x_arr = np.concatenate([dw.normalize_data(x_arr[:,:,:32]), dw.normalize_data(x_arr[:,:,32:])],axis=2)
            x_crops_gen = dw.get_batch_generator(dw.get_crop_generator(x_arr, crop_size, crop_step), buffer)
        else:
            x_arr = np.concatenate([dw.standardize_data(x_arr[:,:,:32]), dw.standardize_data(x_arr[:,:,32:])],axis=2)
            x_crops_gen = (dw.get_batch_generator(dw.get_crop_generator(x_arr[:,:,:32], crop_size, crop_step), buffer),
                           dw.get_batch_generator(dw.get_crop_generator(x_arr[:,:,32:], crop_size, crop_step), buffer))

        # if model has one or two outputs get predicts differently
        res = list()
        if len(load_model.outputs) == 1:
            for x in x_crops_gen:
                res.append(model.predict(x)[:,0])
            res = np.concatenate(res)
        else:
            for time, amp in zip(*x_crops_gen):
                pred = model.predict([time, amp])
                bools = np.squeeze(pred[0], axis=1)
                #depths = np.squeeze(pred[1], axis=1)
                res.append(bools)
            res = np.concatenate(res)

        build = build_pred_map(ex_x_df, orig_shape, res, 
                    crop_size, crop_step, xshift=xshift)
        
        yield y_df, ex_y_df, build''';

In [ ]:
'''def make_pred_fast_new(model, 
                   path_to_data_tuple: tuple[str,str,str],
                   crop_step_xshifts,
                   **kvargs):
    """
    Build prediction maps for the model. For given crop steps and
    xshifts (Which are stored in crop_step_xshifts param as dict with
    keys - crop steps values, and values - any iterable with xshifts for
    the crop step).

    Parameters
    ----------
    model: tensorflow.keras.Model
        The tensorflow model to use in prediction.
    path_to_data_tuple: tuple[str,str,str]
        The tuple with the paths for data file like "*_data.csv", 
        like "*_defects.csv" and like "*_pipe.csv".
    **kvargs
        Any additional params that may be defined in the get_xshifts_data_generator().
    """
    x_df, y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path=path_to_data_tuple))
    for crop_step, xshifts in crop_step_xshifts.items():
        yield get_xshifts_data_generator(model, x_df, y_df, xshifts, crop_step, **kvargs)''';

In [ ]:
'''crop_size_xshifts = {crop_step:list(range(XSHIFT,XSHIFT+crop_step,1)) 
                     for crop_step in CROP_STEPS}
# print([type(item) for item in g])
for run in RUNS:
    print(f'run={run}')
    data_gen = make_pred_fast_new(load_model, PATH_TO_DATA[f'run_{run}'], crop_size_xshifts)
    for crop_step_xshift_gen in data_gen: # 
        for y_df, ex_y_df, res_gen in crop_step_xshift_gen:
            for (orig_size_result_df, extend_size_result_df, info) in res_gen:
                print(info, f'{orig_size_result_df.shape=}', f'{extend_size_result_df.shape=}')''';

# Старый алгоритм построения предсказаний

In [ ]:
def make_pred_fast(model, 
                   path_to_data_tuple: tuple(),
                   x_shifts,
                   y_shifts,
                   crop_size: int, 
                   crop_steps): #, thold: float):

    read_x_df, read_y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path=path_to_data_tuple))
    
    for x_shift in x_shifts:
        for y_shift in y_shifts:
            x_df = dw.roll_df(read_x_df, x_shift, 1)
            y_df = dw.roll_df(read_y_df, x_shift, 1)
        
            #or_rows = x_df.shape[0]
            #or_cols = x_df.shape[1]
            
            #ex_x_df = dw.extend_df_for_prediction(x_df, crop_size)
            #ex_x_df = dw.match_df_for_crops_dividing(ex_x_df, crop_size, crop_step)
            #ex_y_df = dw.extend_df_for_prediction(y_df, crop_size)
            #ex_y_df = dw.match_df_for_crops_dividing(ex_y_df, crop_size, crop_step)
            '''ex_x_df = dw.match_df_for_crops_dividing(x_df, crop_size, crop_step)
            ex_y_df = dw.match_df_for_crops_dividing(y_df, crop_size, crop_step)'''
    
            
            #ex_rows = ex_x_df.shape[0]
            #ex_cols = ex_x_df.shape[1]
    
            #x_df = x_df.iloc[40+y_shift:,:] #40 0
            #y_df = y_df.iloc[40+y_shift:,:]
            x_df = x_df.iloc[y_shift:,:]
            y_df = y_df.iloc[y_shift:,:]
            print(f'xshift={x_shift}, yshift={y_shift}. Shape before: {y_df.shape}')
            if x_df.shape[0]%crop_size != 0:
                x_df = x_df.iloc[:-1*(x_df.shape[0]%crop_size)]
                y_df = y_df.iloc[:-1*(y_df.shape[0]%crop_size)]
            print(f'shape after: {y_df.shape}')
            print()
            x_arr = dw.df_to_numpy(x_df)
            y_arr = y_df.to_numpy()
    
            #x_arr = x_arr[41:, :]
            #y_arr = y_arr[41:, :]
    
            #x_arr = x_arr[:-1*(x_arr.shape[0]%crop_size)]
            #y_arr = y_arr[:-1*(y_arr.shape[0]%crop_size)]
        
            # normalize_data
            # standardize_datav
            
            x_arr = np.concatenate([dw.normalize_data(x_arr[:,:,:32]), dw.normalize_data(x_arr[:,:,32:])],axis=2)
            x_crops_gen = dw.get_batch_generator(dw.get_crop_generator(x_arr, crop_size, crop_step), 15000)
        
            # 1 output
            res = list()
            for x in x_crops_gen:
                #x = np.pad(x, ((0,0),(1,1),(1,1),(0,0)), 'reflect')
                res.append(model.predict(x)[:,0])
                #print(model.predict(x)[:,0].shape)
            res = np.concatenate(res)
            #print(res.shape)
            res_it = iter(res)
        
            # 2 outputs
            '''res = np.array(model.predict([x_data_time, x_data_amp]))
            res = np.squeeze(res, axis=2)
            bool_res = res[0,:]
            depth_res = res[1,:] 
            bool_res_it = iter(bool_res) 
            depth_res_it = iter(depth_res) 
            res_it = bool_res_it'''
            
            def fill_by_arrays(df_cell_value):
                return list()
            def add_number_to_arrays(df_cell_value, number):
                df_cell_value.append(number)
                return df_cell_value
                
            extend_size_result_df = pd.DataFrame(data=0.0, 
                                     columns=x_df.columns, 
                                     index=x_df.index)
            
            #extend_size_result_df = extend_size_result_df.map(fill_by_arrays)
            ex_rows = x_arr.shape[0]
            ex_cols = x_arr.shape[1]
            for i in range(0, ex_rows - crop_size + 1, crop_step):
                for j in range(0,  ex_cols - crop_size + 1, crop_step):
                    temp_add = next(res_it)
                    #print(temp_add)
                    #extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                    #   extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: add_number_to_arrays(x, temp_add))
                    #extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = temp_add
                    '''if temp_add >= thold:
                        extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                            extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: x+1)
                    else:
                        extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                            extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: x-1)'''
                    
                    extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = temp_add
        
            '''extend_size_result_df = pd.DataFrame(data=dw.normalize_data(extend_size_result_df.to_numpy()), 
                                                 index=extend_size_result_df.index, 
                                                 columns=extend_size_result_df.columns)'''
                                                 
            #orig_size_result_df = extend_size_result_df.iloc[crop_size-1:, crop_size-1:].iloc[:or_rows, :or_cols]
            #orig_size_result_df = extend_size_result_df.iloc[crop_size:, crop_size:].iloc[:or_rows-1, :or_cols-1]
            #orig_size_result_df = extend_size_result_df.iloc[:or_rows, :or_cols]
    
            
            yield extend_size_result_df, y_df, x_shift, y_shift #orig_size_result_df, extend_size_result_df, y_df, ex_y_df

In [ ]:
'''arr = np.arange(11)
print(arr)
quart = [*np.quantile(arr,[0.25,0.5,0.75]),arr.max()]
print(quart)

# calc hist of elemens of quartiles
def hist(arr: np.ndarray, bounds: list[int], res: list[int]=[]):
    if not bounds or not arr.shape:
        return res
    bound = bounds.pop(0)
    res.append(arr[arr<=bound].size)
    return hist(arr[arr>bound], bounds, res)

print('hists: ', hist(arr, quart))''';

In [ ]:
def calc_expected_value(df_cell):
    arr = np.array(df_cell)
    uniques, counts = np.unique(arr, return_counts=True)
    weights = counts/arr.size
    return (uniques * weights).sum() / weights.sum()

def calc_1_quartile(df_cell):
    return np.quantile(df_cell,[0.25])
def calc_2_quartile(df_cell):
    return np.quantile(df_cell,[0.5])
def calc_3_quartile(df_cell):
    return np.quantile(df_cell,[0.75])
'''def calc_half(df_cell):
    return np.quantile(df_cell,[0.5])'''

In [ ]:
"""# new
for run in RUNS:
    for crop_step in CROP_STEPS:
        xshifts_range = range(XSHIFT,XSHIFT+crop_step,1)
        data_gen = make_pred_fast(load_model, PATH_TO_DATA[f'run_{run}'], xshifts_range, CROP_SIZE, CROP_STEPS)
        for xshift in xshifts_range:
            #logger.debug(f'run: {run}, crop step: {crop_step}')
            (orig_size_result_df, 
            extend_size_result_df,
            orig_size_reference_df, 
            extend_size_reference_df) = next(data_gen)
            
            p_data = 3 # if crop_step >= 8 else 2

            '''bins_arr = [101] #6,11,21,
            if crop_step < 8:
                bind_arr = bins_arr[:3]'''
            
            for bins in [21]: #
                path_to_file = (pathlib.Path(f'images/models predicts')/
                                f'model_id=v{MODEL_VER:02}n{MODEL_NUM:02}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)_epoch={EPOCH}'/
                                f'run_{run}'/'xshifts'/f'crop_step={crop_step}')
                '''if not os.path.exists(path_to_file):
                    os.makedirs(path_to_file)
                dw.draw_defects_map(orig_size_reference_df, 
                    title = f'REFERENCE run = {run}',
                    path_to_save = str(path_to_file/f'REFERENCE run={run}.jpg'))'''

                #bins = 21 if crop_step == 4 else 101
                
                #for func in [max, min, np.mean, calc_half, calc_quartile, calc_expected_value]:
                for func in [calc_1_quartile, calc_2_quartile, calc_3_quartile]: #[max, min, np.mean, calc_1_quartile, calc_2_quartile, calc_3_quartile, calc_expected_value]:
                
                    '''func_path_to_file = None

                    if len(xshifts_range) == 1:
                        func_path_to_file = path_to_file
                    elif len(xshifts_range) == 2:
                        func_path_to_file = path_to_file/func.__name__
                    else:
                        func_path_to_file = path_to_file/func.__name__/f'bins={bins}'''

                    func_path_to_file = path_to_file/func.__name__
                    
                    if not os.path.exists(func_path_to_file):
                        os.makedirs(func_path_to_file)
        
                    or_temp = orig_size_result_df.map(lambda x: func(x))

                    dw.draw_defects_map_with_reference_owerlap(or_temp, orig_size_reference_df, 
                        title = f'OVERLAPPED PREDICTION (xshift={xshift}, bins={bins}, func={func.__name__}, run={run}, crop step={crop_step}, crop size={CROP_SIZE})',
                        polygonize_data=p_data, bins=bins,
                        path_to_save = func_path_to_file/f'OVERLAPPED PREDICTION (xshift={xshift}, bins={bins}, func={func.__name__}, run={run}, crop step={crop_step}, crop size={CROP_SIZE}).png')

                    '''dw.draw_defects_map(or_temp, 
                        title = f'PREDICTION {func.__name__} run: {run}, crop step = {crop_step}, crop size = {CROP_SIZE}',
                        path_to_save = func_path_to_file/f'PREDICTION {func.__name__} run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg')'''
        
                    
                    '''binary = np.arange(0.1,1,0.1)
        
                    for bina in binary:
                        bin_func_path_to_file = func_path_to_file/f'binary_{bina:0.2f}'
                    
                        if not os.path.exists(bin_func_path_to_file):
                            os.makedirs(bin_func_path_to_file)
                    
                        temp = or_temp.map(lambda x: 1 if x >= bina else 0) # or_temp
                        
                        dw.draw_defects_map(temp, 
                            title = f'PREDICTION {func.__name__} binary_{bina:.02f} run = {run}, crop step = {crop_step}, crop size = {CROP_SIZE}',
                            path_to_save = str(bin_func_path_to_file/
                                f'PREDICTION {func.__name__} binary_{bina:.02f} run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))
                    
                        #bin_func_path_to_file = bin_func_path_to_file/f'OVERLAPPED' #f'{func_path_to_file}/binary_{bina}'
                    
                        #if not os.path.exists(bin_func_path_to_file):
                        #    os.makedirs(bin_func_path_to_file)
                        
                        dw.draw_defects_map_with_reference_owerlap(temp, orig_size_reference_df, 
                            title = f'OVERLAPPED PREDICTION {func.__name__} binary_{bina:.02f} run = {run},'+
                               f' crop step = {crop_step}, crop size = {CROP_SIZE}',
                            path_to_save = str(bin_func_path_to_file/
                               f'OVERLAPPED PREDICTION {func.__name__} binary_{bina:.02f} run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))''';
""";

In [ ]:
#for (thold, name) in [(0.1, 'best_recall'), (0.28, 'best_mean_values'), (0.4,'best_precision')]:
for run in RUNS:
    for crop_step in CROP_STEPS:
        xshifts_range = range(XSHIFT,XSHIFT+crop_step,1)
        yshifts_range = range(8) #15 8 4
        data_gen = make_pred_fast(load_model, PATH_TO_DATA[f'run_{run}'], xshifts_range, yshifts_range, CROP_SIZE, CROP_STEPS)
        for (orig_size_result_df, orig_size_reference_df, xshift, yshift) in data_gen:
            #logger.debug(f'run: {run}, crop step: {crop_step}')
            #orig_size_result_df, orig_size_reference_df, x_shift, y_shift = next(data_gen)
            
            #bins_arr = [101, 1001]

            #for bins in bins_arr:
            path_to_file = (pathlib.Path(f'data/generated_content/CNN/in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)/')/
                            f'model_id=v{MODEL_VER:04}n{MODEL_NUM:04}_epoch={EPOCH}'/
                            'test+val'/f'crop_step={crop_step}') # /f'bins={bins}' /'xshifts' f'run_{run}'
            
            if not os.path.exists(path_to_file):
                os.makedirs(path_to_file)
            # bins=bins, 
            dw.draw_defects_map_with_reference_owerlap(orig_size_result_df, orig_size_reference_df, 
                title = f'OVERLAPPED PREDICTION (xshift={xshift}, yshift={yshift}, run={run}, crop step={crop_step}, crop size={CROP_SIZE})',
                polygonize_data=3, 
                path_to_save = path_to_file/f'OVERLAPPED PREDICTION (xshift={xshift}, yshift={yshift}, run={run}, crop step={crop_step}, crop size={CROP_SIZE}).png')

In [ ]:
'''arr = np.arange(34)
print(arr)
print(arr.shape[0]%16)
print(arr[:-1*(arr.shape[0]%16)])
23%16'''

In [ ]:
"""# old
#for (thold, name) in [(0.1, 'best_recall'), (0.28, 'best_mean_values'), (0.4,'best_precision')]: #np.arange(0.1,1,0.1):
for xshift in range(200,217,1):
    for run in RUNS:
        for crop_step in CROP_STEPS:
            #logger.debug(f'thold: {thold}, run: {run}, crop step: {crop_step}')
            (orig_size_result_df, 
            extend_size_result_df,
            orig_size_reference_df, 
            extend_size_reference_df) = make_pred_fast(load_model, PATH_TO_DATA[f'run_{run}'], xshift, CROP_SIZE, crop_step)#, thold)
    
            path_to_file = (pathlib.Path('images/models predicts')/
                            f'model_id=v{MODEL_VER:02}n{MODEL_NUM:02}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)_epoch={EPOCH}'/
                            f'run_{run}'/f'xhifts') 
    
            if not os.path.exists(path_to_file):
                os.makedirs(path_to_file)
            
            dw.draw_defects_map(orig_size_reference_df, 
                title = f'REFERENCE run = {run}, xhift={xshift}',
                path_to_save = str(path_to_file/f'xhift={xshift}_REFERENCE run={run}.jpg'))
            
            #path_to_file = path_to_file/'old_math_reduce(add_or_remove=1)'/f'trashhold={thold:0.1f}({name})'
            #path_to_file = path_to_file/'old_math_reduce(add_model_pred)'
            
            #if not os.path.exists(path_to_file):
            #    os.makedirs(path_to_file)
            
            dw.draw_defects_map(orig_size_result_df, 
                title = f'PREDICTION run: {run}, crop step = {crop_step}, crop size = {CROP_SIZE}, xhift={xshift}')#,
                #path_to_save = str(path_to_file/f'xhift={xshift}_PREDICTION run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))'''
                
            dw.draw_defects_map_with_reference_owerlap(orig_size_result_df, orig_size_reference_df, 
                title = f'OVERLAPPED PREDICTION run: {run}, crop step = {crop_step}, crop size = {CROP_SIZE}, xhift={xshift}', 
                annotate_data=True, annotate_font_size=crop_step)#,
                #path_to_save = str(path_to_file/f'xhift={xshift}_OVERLAPPED PREDICTION run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))
    
            binary = np.arange(0.1,1,0.1)
    
            for bina in binary:
                bin_func_path_to_file = path_to_file/f'binary_{bina:0.2f}'
            
                if not os.path.exists(bin_func_path_to_file):
                    os.makedirs(bin_func_path_to_file)
    
                temp = orig_size_result_df.map(lambda x: 1 if x >= bina else 0) # or_temp
                
                dw.draw_defects_map(temp, 
                    title = f'PREDICTION binary_{bina:.02f} run = {run}, crop step = {crop_step}, crop size = {CROP_SIZE}',
                    path_to_save = str(bin_func_path_to_file/
                                f'PREDICTION binary_{bina:.02f} run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))
    
                #bin_func_path_to_file = bin_func_path_to_file/f'OVERLAPPED' #f'{func_path_to_file}/binary_{bina}'
            
                #if not os.path.exists(bin_func_path_to_file):
                #    os.makedirs(bin_func_path_to_file)
                
                dw.draw_defects_map_with_reference_owerlap(temp, orig_size_reference_df, 
                    title = f'OVERLAPPED PREDICTION binary_{bina:.02f} run = {run}, crop step = {crop_step}, crop size = {CROP_SIZE}',
                    path_to_save = str(bin_func_path_to_file/
                                   f'OVERLAPPED PREDICTION binary_{bina:.02f} run={run}, crop step={crop_step}, crop size={CROP_SIZE}.jpg'))""";